In [ ]:
##############################################
##### PARTIE 1 : IMPORTATION DES DONNEES #####
##############################################

In [ ]:
# Dans cette partie, on importe des données relatives aux risques (pollutions industrielles, inondations, qualité de l'air, sites pollués) et le tableau descriptif
# des communes tel que mis à disposition par l'INSEE.
# L'objectif est de faire la jointure est ces deux types de sources.

In [2]:
# Importation des modules utiles pour cette partie

import requests
import zipfile

import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
import contextily as ctx

In [63]:
#### On importe le fichier répertoriant les installations industrielles rejetant des polluants, disponible sur Géorisques

# On télécharge le fichier avec la localisation des installations industrielles rejetant des polluants, directement depuis le site Géorisques
url_industries_polluantes = "https://files.georisques.fr/irep/2023.zip"
response_industries_polluantes = requests.get(url_industries_polluantes)

# On enregistre le dossier localement
with open("Industries_polluantes.xlsx", "wb") as f:
    f.write(response_industries_polluantes.content)

# On importe le fichier répertoriant les établissements polluants (qui est une liste des établissements)
with zipfile.ZipFile("Industries_polluantes.xlsx", "r") as z:
    with z.open("2023/etablissements.csv") as f:
        industries_polluantes = pd.read_csv(f, sep=None, engine='python')

# On renomme la colonne insee dès maintenant (pour harmoniser ensuite)
industries_polluantes = industries_polluantes.rename(columns={"code_insee" : "insee"})
industries_polluantes.head()

# On importe le fichier des rejets (pour chaque établissement listé dans industries_polluantes)
with zipfile.ZipFile("Industries_polluantes.xlsx", "r") as z:
    with z.open("2023/rejets.csv") as f:
        rejets = pd.read_csv(f, sep=None, engine='python')
rejets = rejets.rename(columns = {"code_insee" : "insee"})
rejets.head()

# On fusionne les deux tables
etablissements_rejets = pd.merge(industries_polluantes, rejets, on=["identifiant", "nom_etablissement", "code_postal", "insee",
"commune", "code_departement", "departement", "code_region", "region"], how="inner")

# On obtient les variables d'intérêt suivantes :
# nom de l'établissement (pour le versant établissements);
# année des rejets, total des rejets raccordés au réseau, total des rejets isolés (pour le versant rejets)
etablissements_rejets.columns

# On regroupe par la commune et on enregistre les pollutions (total des rejets ET nombre d'établissements)
etablissements_rejets_agr = etablissements_rejets.groupby("insee").agg(
    rejets_isoles=("rejet_isole_m3_par_an", "sum"),
    nb_etablissements_isoles=("rejet_isole_m3_par_an", "count"),
    rejets_raccordes=("rejet_raccorde_m3_par_an", "sum"),
    nb_etablissements_raccordes=("rejet_raccorde_m3_par_an", "count")).reset_index()

# On fait, pour chaque commune, la somme des rejets raccordés et isolés pour les deux colonnes concernées
etablissements_rejets_agr["rejets_tot"] = etablissements_rejets_agr["rejets_isoles"] + etablissements_rejets_agr["rejets_raccordes"]
etablissements_rejets_agr["nb_etablissements_tot"] = etablissements_rejets_agr["nb_etablissements_isoles"] + etablissements_rejets_agr["nb_etablissements_raccordes"]

In [64]:
#### On importe le fichier avec les shapefiles des communes

# Téléchargement du fichier avec les shapefiles des communes, directement depuis Datagouv
url_communes = "https://www.data.gouv.fr/api/1/datasets/r/0e117c06-248f-45e5-8945-0e79d9136165"
zip_communes = "shapefiles.zip"

# Enregistrement du fichier localement
response_communes = requests.get(url_communes)
with open(zip_communes, "wb") as f:
    f.write(response_communes.content)

# Importation avec GeoPandas
communes = gpd.read_file(f"zip://{zip_communes}!communes-20220101.shp")
communes.head()

,insee,nom,wikipedia,surf_ha,geometry
0,2B222,Pie-d'Orezza,fr:Pie-d'Orezza,573.0,"POLYGON ((9.32017 42.38507, 9.32028 42.3851, 9..."
1,2B137,Lano,fr:Lano,824.0,"POLYGON ((9.2001 42.39013, 9.20014 42.39014, 9..."
2,2B051,Cambia,fr:Cambia,833.0,"POLYGON ((9.27757 42.37509, 9.27758 42.37512, ..."
3,2B106,Érone,fr:Érone,393.0,"POLYGON ((9.2512 42.37605, 9.25132 42.37603, 9..."
4,2B185,Oletta,fr:Oletta,2674.0,"POLYGON ((9.2834 42.66273, 9.28345 42.66273, 9..."


In [65]:
### On fait une première fusion entre les communes et les pollutions industrielles

# On fait la jointure (en ne conservant que les communes qui sont dans les deux df, ie celles qui ont des rejets identifiés)
communes_rejets = pd.merge(communes, etablissements_rejets_agr, on="insee", how = "inner")

# Conversion du fichier au format GeoPandas
communes_rejets = gpd.GeoDataFrame(communes_rejets, geometry='geometry')

# On crée un df avec les communes et une variable polluée ou non (binaire)
communes_statut = communes.copy()
communes_statut["Polluee"] = communes_statut["insee"].isin(communes_rejets["insee"])

# On regarde combien on en a (2 375 polluées parmi 34 955 : environ 7%)
print(communes_statut["Polluee"].sum(), communes.shape)

2375 (34955, 5)


In [66]:
# Avant de fusionner etablissements_rejets et communes_statut, on vérifie qu'il n'y a pas d'incohérence
print(
    (set(etablissements_rejets["insee"].astype(str)) - set(communes_statut["insee"].astype(str))),
    (set(communes_statut["insee"].astype(str)) - set(etablissements_rejets["insee"].astype(str)))
    )

# On supprime les codes en trop
codes_pollution = set(etablissements_rejets["insee"].astype(str))
codes_communes  = set(communes_statut["insee"].astype(str))
codes_non_trouves = codes_pollution - codes_communes
codes_arrondissements = {c for c in codes_non_trouves if c.startswith(("75", "13", "69"))}
etablissements_rejets = etablissements_rejets[
    ~etablissements_rejets["insee"].astype(str).isin(codes_non_trouves - codes_arrondissements)
]


{'13216', '75115', '27676', '85212', '50354', '13208', '13205', '01154', '16274', '69389', '49039', '13211', '13210', '85217', '74268', '69383', '33267', '61356', '69387', '49274', '75111', '13215', '13214', '13203', '79264', '49276', '69384', '75109', '69388', '73002', '75101', '13209', '49204', '50163'} {'81023', '25395', '60677', '51450', '65371', '56056', '17348', '51206', '76594', '92033', '89171', '74178', '54238', '41072', '67455', '27289', '19053', '04144', '68106', '08499', '19150', '90069', '02785', '57061', '06059', '28009', '95574', '76014', '02145', '30065', '2B046', '24331', '58104', '73294', '53235', '44091', '60079', '28148', '12011', '55544', '02089', '14242', '30257', '24349', '80023', '70149', '04184', '32458', '56173', '60643', '27545', '52025', '06151', '89196', '85304', '54140', '24141', '53052', '33261', '59209', '29016', '62323', '50121', '87111', '16047', '09025', '44206', '32224', '58239', '61450', '08033', '80386', '43206', '59018', '66055', '73142', '71281',

In [67]:
# On fusionne pour Paris, Lyon et Marseille (dans une colonne à part au cas où)
etablissements_rejets["insee_agr"] = etablissements_rejets["insee"]
etablissements_rejets["insee_agr"] = etablissements_rejets["insee_agr"].replace(["13203", "13205", "13208", "13209", "13210", "13211", "13214", "13215", "13216"],
"13055").astype(str)
etablissements_rejets["insee_agr"] = etablissements_rejets["insee_agr"].replace(["69383", "69384", "69387", "69388", "69389"],"69123").astype(str)
etablissements_rejets["insee_agr"] = etablissements_rejets["insee_agr"].replace(["75101", "75109", "75111", "75112", "75115"],"75056").astype(str)

In [68]:
# On fusionne etablissements_rejets_agr et communes_statut (ce qui réduit communes_statut aux communes polluées)
communes_statut = pd.merge(communes_statut, etablissements_rejets_agr, on = "insee", how = "left")

In [ ]:
# On ne conserve que les villes qui ont un rejet isolé positif
#communes_statut = communes_statut[communes_statut["rejets_tot"] > 0]

In [69]:
#### On importe le dossier complet de l'INSEE pour apparier avec des données "sociales"

# On télécharge le dossier qui contient notre csv
url_complet = "https://www.insee.fr/fr/statistiques/fichier/5359146/dossier_complet.zip"
zip_path = "dossier_complet.zip" 

# On enregistre le fichier localement
response = requests.get(url_complet)
with open(zip_path, "wb") as f:
    f.write(response.content)

# On importe notre fichier avec Pandas (et on l'appelle complet_insee)
csv_complet_insee = "dossier_complet.csv" 
with zipfile.ZipFile(zip_path) as z:
    with z.open(csv_complet_insee) as f:
        complet_insee = pd.read_csv(f, sep=';', encoding='utf-8')

/tmp/ipykernel_58248/1042689086.py:16: DtypeWarning: Columns (0,1701,1702) have mixed types. Specify dtype option on import or set low_memory=False.
  complet_insee = pd.read_csv(f, sep=';', encoding='utf-8')


In [81]:
#### On fait à nouveau la jointure avec notre df précédent

# On renomme la variable CODGEO en "insee" pour que ce soit homogène
complet_insee = complet_insee.rename(columns={"CODGEO":"insee"})

# On s'assure que la variable "insee" est similaire dans les deux tableaux
communes_statut['insee'] = communes_statut['insee'].astype(str).str.zfill(5)    # zfill remplit de zéros à gauche pour atteindre 5 signes (pour harmoniser à la norme INSEE)
complet_insee['insee'] = complet_insee['insee'].astype(str).str.zfill(5)

# On fusionne
communes_complet = pd.merge(communes_statut, complet_insee, on="insee") # on obtient un tableau à 1987 colonnes (et 34 931 lignes)
communes_complet.shape

(34931, 1987)

In [82]:
#### On importe le fichier répertoriant les catastrophes naturelles de type inondations reconnues pour chaque commune

# Téléchargement du fichier avec le nombre de catastrophes reconnues pour chaque commune, directement depuis le site Géorisques
url_inondations = "https://files.georisques.fr/onrn/2025/ONRN_Reco_INON_8224.xlsx"
response_inondations = requests.get(url_inondations)

# Enregistrement du fichier localement
with open("ONRN_Reco_INON_8224.xlsx", "wb") as f:
    f.write(response_inondations.content)

# Importation et ouverture avec Pandas
inondations = pd.read_excel("ONRN_Reco_INON_8224.xlsx", sheet_name="Nb reco. Cat Nat")

# On renomme les colonnes
inondations.columns = ['insee', 'Commune', 'Occurrences'] 

In [83]:
# On remarque que communes_complet prend en compte des communes en outre-mer (et pas inondations)
set(communes_complet["insee"].astype(str)) - set(inondations["insee"].astype(str))

# On les supprime
codes_complet = set(communes_complet["insee"].astype(str))
codes_inondations  = set(inondations["insee"].astype(str))
codes_a_supprimer = codes_complet - codes_inondations
communes_complet = communes_complet[
    ~communes_complet["insee"].astype(str).isin(codes_a_supprimer)
]

In [86]:
# On remarque, dans l'autre sens, que inondations prend en compte des communes qui n'existe plus (et qui ne sont pas dans communes_complet)
set(inondations["insee"].astype(str)) - set(communes_complet["insee"].astype(str))

# On les supprime également, avant de faire la jointure
codes_a_supprimer_2 = codes_inondations - codes_complet
inondations = inondations[
    ~inondations["insee"].astype(str).isin(codes_a_supprimer_2)
]

In [88]:
### Préparation du dataframe de travail

# On s'assure qu'on a bien le même nombre de lignes
print(inondations.shape, communes_complet.shape)

# On fait la jointure
communes_complet = pd.merge(communes_complet, inondations, on="insee")

(34802, 3) (34802, 1987)


In [90]:
# On remplace "Pas de reconnaissance" par 0 (on transforme la ligne en numérique : les erreurs (ie les caractères) sont alors convertis en 0)
communes_complet["Occurrences"] = pd.to_numeric(communes_complet["Occurrences"], errors="coerce").fillna(0)

In [ ]:
### On se concentre sur l'idf

# On crée un df communes seulement avec l'idf
communes["departement"] = communes["insee"].astype(str).str.slice(0,2)
communes_idf = communes.copy()
communes_idf = communes_idf[communes_idf["departement"].isin(["75", "77", "78", "91", "92", "93", "94", "95"])]

communes_polluees["departement"] = communes_polluees["insee"].astype(str).str.slice(0,2)
communes_polluees_idf = communes_polluees.copy()
communes_polluees_idf = communes_polluees_idf[communes_polluees_idf["departement"].isin(["75", "77", "78", "91", "92", "93", "94", "95"])]

# On crée un df avec les communes d'IDF et une variable polluée ou non (binaire)
communes_idf_statut = communes_idf.copy()
communes_idf_statut["Polluee"] = communes_idf["insee"].isin(communes_polluees_idf["insee"])

# On regarde combien on en a (157 polluées parmi 1 268 : environ 10% (c'est cool !))
print(communes_idf_statut["Polluee"].sum(), communes_idf.shape)